# Fundamental Properties of EMU Radio Sources - Cross-Match Catalogs using NWAY

## Import Libraries

In [4]:
from astropy.io import fits
import numpy as np

## Import Catalog Data

In [ ]:
emu_catalog_path = ''
des_catalog_path = ''
viking_catalog_path = ''
catwise_catalog_path = ''

## Prepare the catalogs for NWAY input
The NWAY library has multiple requirements for a catalog to serve as input.
1. The fits data table needs to have a extension name. It is used as a prefix for the columns copied to the output catalog, so each column from each catalog is called "PREFIXOFCATALOG_NAMEOFCOLUMN"
2. The fits header of the catalogs needs to have the SKYAREA keyword. This keyword has the area on the sky in square degrees covered by the catalog. Since I don't have the documentation for EMU data, I must calculate the area using the mosaic image.
3. The position error needs to be a column with a single value, called positional error, so we need to obtain it from the right ascension and declination errors.

### Change extension name
This process is also possible using just the NWAY library. Check page 8 of the NWAY documentation

In [41]:
def change_hdu_name(fits_file, hdu_index, new_name):
    with fits.open(fits_file, mode='update') as hdul:
        # Change the name of the specified HDU
        hdul[hdu_index].header['EXTNAME'] = new_name
        # Save changes to the FITS file
        hdul.flush()

Change extension name of EMU survey to "EMU" and of GAMA-23 survey to "G23"

In [ ]:
change_hdu_name(emu_catalog_path, hdu_index=1, new_name="EMU")
change_hdu_name(des_catalog_path, hdu_index=1, new_name="DES")
change_hdu_name(viking_catalog_path, hdu_index=1, new_name="VKG")
change_hdu_name(catwise_catalog_path, hdu_index=1, new_name="CAT")

### Create and assign value to SKYAREA keyword

Create the SKYAREA in the FITS header and assign a value to it.

In [44]:
def create_skyarea_keyword(fits_file, skyarea_value):
    """
    Create the SKYAREA keyword in the FITS header and assign a value to it.

    Parameters:
    fits_file (str): Path to the FITS file.
    skyarea_value (float): Value to assign to the SKYAREA keyword (in square degrees).
    """
    with fits.open(fits_file, mode='update') as hdul:
        header = hdul[1].header
        
        header['SKYAREA'] = skyarea_value
        
        hdul.flush()
        print(f"SKYAREA keyword set to {skyarea_value} in {fits_file}.")

In [7]:
def calculate_skyarea(ra_interval, dec_interval):
    """
    Calculate the area of the sky.

    Parameters:
    ra_interval (list): List of interval of RA in degrees
    dec_interval (list): List of interval of DEC in degrees

    """
    delta_ra_rad  = np.radians(abs(ra_interval[1]-ra_interval[0]))
    delta_dec_rad = np.sin(np.radians(dec_interval[1])) - np.sin(np.radians(dec_interval[0]))

    solid_angle_sr = delta_ra_rad*delta_dec_rad
    solid_angle_deg2 = solid_angle_sr * (180 / np.pi) ** 2

    return solid_angle_deg2

In [10]:
survey_area = calculate_skyarea([11.75,19.5], [-35.50,-29.50])
print(survey_area)

39.19978509362287


In [ ]:
area_emu_survey = calculate_skyarea([11.75,19.5], [-35.50,-29.50])

area_des_survey = calculate_skyarea([11.75,19.5], [-35.50,-29.50])
area_viking_survey = calculate_skyarea([11.75,19.5], [-35.50,-29.50])
area_catwise_survey = calculate_skyarea([11.75,19.5], [-35.50,-29.50])


create_skyarea_keyword(emu_catalog_path, area_emu_survey)
create_skyarea_keyword(des_catalog_path, area_des_survey)
create_skyarea_keyword(viking_catalog_path, area_viking_survey)
create_skyarea_keyword(catwise_catalog_path, area_catwise_survey)

SKYAREA keyword set to 82.7 in /home/polaris/Lab_Astro/data/EMU_data/G23-ASKAP-EMUES-master-cat.fits.
SKYAREA keyword set to 50.6 in /home/polaris/Lab_Astro/data/GAMA_data/gkvInputCatv02.fits.


### Create positional error column and append to fits file
This function considers that both the error in RA and DEC are in angular units. The positional error is calculated by using the uncertainty formula.

In [46]:
def create_positional_error_column(fits_file):
    with fits.open(fits_file, mode='update') as hdul:
        data = hdul[1].data

        ra_error = data['E_RA']
        dec_error = data['E_DEC']

        # Compute the positional error
        pos_error_data = np.sqrt(ra_error ** 2 + dec_error ** 2) *3600  # Convert from degrees to arcseconds

        # Check if the column 'POS_ERROR' already exists
        if 'POS_ERROR' in data.columns.names:
            # Update the existing column
            data['POS_ERROR'] = pos_error_data
            print("Updated existing 'POS_ERROR' column with new values.")
        else:
            # Create a new column
            pos_error_col = fits.Column(name='POS_ERROR', format='D', array=pos_error_data)  # 'D' is for double precision float (64-bit)

            # Add the new column to the existing table
            new_columns = hdul[1].columns + fits.ColDefs([pos_error_col])
            new_hdu = fits.BinTableHDU.from_columns(new_columns)

            # Replace the binary table HDU with the updated one
            hdul[1] = new_hdu
            print("Created new 'POS_ERROR' column and added it to the table.")

        # Save changes to the FITS file
        hdul.flush()

In [47]:
create_positional_error_column(emu_catalog_path)

Updated existing 'POS_ERROR' column with new values.


## Fix ID Column in EMU survey

In [48]:

def create_id_column_from_source_name(fits_file):
    with fits.open(fits_file, mode='update') as hdul:
        data = hdul[1].data

        if 'Source_Name' not in data.columns.names:
            raise ValueError("The FITS file does not contain a 'Source_Name' column.")
        
        if 'ID' in data.columns.names:
            raise ValueError("The FITS file already contains an 'ID' column.")

        source_names = data['Source_Name']

        id_data = [name.split('EMUJ')[-1] if 'EMUJ' in name else '' for name in source_names]

        id_col = fits.Column(name='ID', format='20A', array=id_data)  # '20A' for a string column of max length 20

        new_columns = hdul[1].columns + fits.ColDefs([id_col])
        new_hdu = fits.BinTableHDU.from_columns(new_columns)

        hdul[1] = new_hdu

        hdul.flush()
        print("Created new 'ID' column with the number part of 'Source_Name'.")

In [49]:
create_id_column_from_source_name(emu_catalog_path)

ValueError: The FITS file already contains an 'ID' column.